<a href="https://colab.research.google.com/github/rodrigofer89/Machine-Learning/blob/main/Regress%C3%A3o_Linear_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Entendimento do problema**

É possível prever a altura dos filhos baseado nas alturas dos pais?

In [38]:
# Imports necessários
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [39]:
df = pd.read_csv('galton.csv')
df.head(5)

,family,father,mother,midparentHeight,children,childNum,gender,childHeight
0,1,78.5,67.0,75.43,4,1,male,73.2
1,1,78.5,67.0,75.43,4,2,female,69.2
2,1,78.5,67.0,75.43,4,3,female,69.0
3,1,78.5,67.0,75.43,4,4,female,69.0
4,2,75.5,66.5,73.66,4,1,male,73.5


In [40]:
# Quantos dados?
df.shape

(934, 8)

In [41]:
# Quais os tipos? Dados faltantes?
# não existem dados faltantes
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 934 entries, 0 to 933
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   family           934 non-null    object 
 1   father           934 non-null    float64
 2   mother           934 non-null    float64
 3   midparentHeight  934 non-null    float64
 4   children         934 non-null    int64  
 5   childNum         934 non-null    int64  
 6   gender           934 non-null    object 
 7   childHeight      934 non-null    float64
dtypes: float64(4), int64(2), object(2)
memory usage: 58.5+ KB


In [ ]:
# A coluna familia aparece como string pois existem valores alfabéticos, não será uma coluna útil
df['family'].values

In [43]:
# Alguma info estatística útil?
df.describe()

,father,mother,midparentHeight,children,childNum,childHeight
count,934.000000,934.000000,934.000000,934.000000,934.000000,934.000000
mean,69.197109,64.089293,69.206773,6.171306,3.585653,66.745931
std,2.476479,2.290886,1.802370,2.729025,2.361410,3.579251
min,62.000000,58.000000,64.400000,1.000000,1.000000,56.000000
25%,68.000000,63.000000,68.140000,4.000000,2.000000,64.000000
50%,69.000000,64.000000,69.248000,6.000000,3.000000,66.500000
75%,71.000000,65.875000,70.140000,8.000000,5.000000,69.700000
max,78.500000,70.500000,75.430000,15.000000,15.000000,79.000000


In [44]:
# Quantos valores únicos por coluna?
df.nunique()

family             205
father              35
mother              29
midparentHeight    140
children            12
childNum            15
gender               2
childHeight         67
dtype: int64

**Limpeza dos dados**

In [45]:
# Removendo duplicatas, nesse caso não axistem dados duplicados
df.drop_duplicates(inplace=True)

In [46]:
# Removendo dados faltantes, nesse caso não existem dados faltantes
df.dropna(inplace=True)

In [47]:
df.shape

(934, 8)

Vamos trabalhar com dados em **m**?

In [48]:
def inch2m(inch):
  return inch*2.54/100

In [49]:
inch2m(78.5)

1.9939000000000002

In [50]:
# Aplicando a função para colocar os valores em metros
df['father'] = df['father'].apply(inch2m)

In [51]:
# Fazendo um for nas outras colunas de altura para metros
colunas = ['mother', 'midparentHeight', 'childHeight']
for coluna in colunas:
  df[coluna] = df[coluna].apply(inch2m)

Sabemos que os modelos só trabalham com números, então não podemos deixar a coluna GENDER como está! Precisamos transformá-la!

In [52]:
# Criando uma função para transformar MALE E FEMALE em 0 e 1
def sex(x):
  if x == 'male':
    return 0
  else:
    return 1

In [53]:
# Aplicando a função sex na coluna GENDER
df['gender'] = df['gender'].apply(sex)

In [54]:
# Removendo colunas que não serão úteis
df2 = df.drop(columns=['family', 'childNum', 'children'])

**Modelagem**

In [55]:
# Separação de DADOS e LABEL => y = a*X + b
X = df2.drop(columns=['childHeight'])
y = df2['childHeight']

In [56]:
# Dividindo dados para TREINO e TESTE
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                    test_size = 0.3,
                                    random_state=43)

In [57]:
# Treinando o modelo
modelo = LinearRegression()
modelo.fit(X_train, y_train)

LinearRegression()

In [58]:
# Fazendo as predições
y_pred = modelo.predict(X_test)

**Avaliação das previsões**

In [59]:
# Erro quadrático médio
mean_squared_error(y_test, y_pred)

0.003095344698336929

In [60]:
# Erro absoluto médio
mean_absolute_error(y_test, y_pred)

0.04398745830307559

In [61]:
# R2_score
r2_score(y_test, y_pred)

0.6137640207302077

**Interpretando os coeficientes encontrados!**

Vamos dar uma olhada nos coeficientes encontrados para cada caracteristica:


In [63]:
modelo.intercept_

0.4841855410126339

In [64]:
modelo.coef_

array([-7.10165826e+12, -7.66979092e+12,  1.42033165e+13, -1.31347656e-01])

In [67]:
pd.DataFrame({
        'features': X_train.columns,
        'coeficientes': modelo.coef_
    }).sort_values(by='coeficientes', ascending=False)

,features,coeficientes
2,midparentHeight,1.420332e+13
3,gender,-1.313477e-01
0,father,-7.101658e+12
1,mother,-7.669791e+12
